<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course5/week2/Emojify!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emojify! 

2주차 두 번쨰 과제에 오신 여러분을 환영합니다! 이 과제에서 여러분은 단어 벡터 표현을 사용해서 Emojifier(이모지 변형 알고리즘)을 만들 것입니다.

카카오톡과 같은 텍스트 메시지를 보낼때, 더 예쁘게 보내고 싶지 않나요? 지금부터 만들 emojifier 앱은 이 작업을 도와줍니다. 따라서 아래와 같은 텍스트를 작성하기보단,
> "Congratulations on the promotion! Let's get coffee and talk. Love you!"

다음과 같이 자동으로 이모지로 변경된 텍스트를 작성할 수 있습니다.

> "Congratulations on the promotion! 👍 Let's get coffee and talk. ☕️ Love you! ❤️"

- 특정 문장(예를 들어, "Let's go see the baseball game tonight!")을 입력으로 받아서, 가장 적절한 이모지(위의 예시에서, ⚾️)를 자동으로 찾아주는 모델을 구현합니다.

#### Using word vectors to improve emoji lookups

- 대부분의 이모지 인터페이스에서, ❤️이모지는 "love"라는 심볼보다는 "heart"라는 심볼로 표현된다는 것을 기억해야 합니다.
  - 다시 말해, 위의 이모지를 얻기 위해서는 "heart"라는 텍스트를 타이핑해야 하며, "love"를 타이핑하면 해당 이모지가 나오지 않는다는 것을 기억해야 합니다.
- word vectors를 사용하면 보다 유연한 이모지 인터페이스를 만들 수 이습니다.
- word vector를 사용할 때, 여러분의 훈련 데이터 세트가 특정 이모지의 일부와만 일치한다고 하더라도, 여러분이 구현한 알고리즘은 추가적인 여러 단어들의 연관성을 찾아 일반화하여, 같은 이모지를 얻을 수 있도록 합니다.
  - 이 작업은 훈련 데이터 세트에 없는 단어에 대해서도 작동합니다.
  - 이는 작은 훈련 데이터 셋만을 가지고도 문장을 이모지로 변환시켜주는 정확한 알고리즘을 만들 수 있도록 해줍니다.

#### What you'll build

1. 이번 과제에서, 여러분은 word embedding을 사용한 기본적인 모델(Emojifier-V1)을 만듭니다.
2. 이후 LSTM 알고리즘과 통합하여 (1)번 모델보다 보다 정교한 모델(Emojifier-V2)를 만듭니다.

이제 시작해봅시다! 아래 코드를 실행시켜, 필요한 패키지를 불러옵니다.

In [ ]:
import numpy as np
from emo_utils import *
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

## 1 - Baseline model: Emojifier-V1

### 1.1 - Dataset EMOJISET

가장 기본적인 이모지 분류기를 만들어보겠습니다.

이번에 제공되는 데이터셋 (X,Y)는 꽤 작은 데이터셋입니다.
- X는 127개의 문장(문자열)을 담고 있습니다
- Y는 0부터 4까지 각 문장에 대응되는 이모지를 담고 있습니다.

<img src="arts/data_set.png" style="width:700px;height:300px;">
<center>그림 1: EMOJISET - 
5 개 클래스의 분류 문제. 여기에 몇 가지 문장에 대한 예시가 나와 있습니다.</center>


아래 코드를 사용하여 데이터 세트를 불러와 보겠습니다. 훈련 (127 개 예시)과 테스트 (56 개 예시)로 데이터 세트를 분할했습니다.

In [ ]:
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')

In [ ]:
maxLen = len(max(X_train, key=len).split())

다음 셀을 실행하여 X_train의 문장과 Y_train의 해당 레이블을 인쇄합니다.

* 다른 예를 보려면 `idx`를 변경하십시오.
* iPython 노트북에서 사용하는 글꼴로 인해 하트 이모티콘이 빨간색이 아닌 검은 색으로 표시 될 수 있습니다.

In [ ]:
for idx in range(10):
    print(X_train[idx], label_to_emoji(Y_train[idx]))

### 1.2 - Overview of the Emojifier-V1

이 파트에서, 여러분은 "Emojifer-v1"이라는 기본적인 모델을 구현합니다.

<img src="arts/image_1.png" style="width:900px;height:300px;">
<center>그림 2 : Baseline model(Emojifier-V1)</center>

#### Inputs and outputs
- 이 모델의 입력은 특정 문장을 나타내는 문자열입니다(예를 들어, "I love you")
- 출력은 shape가 (1,5)인 확률 벡터입니다. 따라서 선택할 수 있는 이모지의 종류로 총 5개가 있습니다.
- (1,5) shape의 확률 백터는 argmax 레이어를 통과합니다. 이를 통해 가장 높은 확률을 가진 이모지를 선택할 수 있습니다.

#### One-hot encoding
- 라벨링된 데이터를 softmax 분류 함수에 적합하게 맞추기 위해서, $(m, 1)$의 shape를 하고 있는 $Y$ 데이터를 $(m, 5)$ 형태의 one-hot 표현 방식으로 변환합니다.
  - 각 행은 데이터 셋 한 개의 레이블을 제공하는 one-hot 벡터입니다.
  - 여기서 `Y_oh`는 변수 이름 `Y_oh_train`및 `Y_oh_test`에서 "Y-one-hot"을 의미합니다.

In [ ]:
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)

`convert_to_one_hot()` 함수가 무엇을 하는지 살펴 보겠습니다. 다른 값을 인쇄하려면 `index`를 자유롭게 변경하십시오.

In [ ]:
idx = 50
print(f"Sentence '{X_train[50]}' has label index {Y_train[idx]}, which is emoji {label_to_emoji(Y_train[idx])}", )
print(f"Label index {Y_train[idx]} in one-hot encoding format is {Y_oh_train[idx]}")

이제 모든 데이터를 Emojify-V1 모델에 입력 할 준비가되었습니다. 모델을 구현합시다!

### 1.3 - Implementing Emojifier-V1

그림 2에서 볼 수 있듯이, 첫 번째 단계는,
- 입력 문장의 각 단어를 word vector 표현 방식으로 변환합니다.
- 이후 word vectors의 평균을 계산합니다.
- 이전 과제와 동일하게, 우리는 미리 훈련된 50 차원의 GloVe embedding을 사용합니다.

아래 코드를 실행시켜 단어에 대한 모든 벡터 표현을 담고 있는 `word_to_vec_map`을 불러옵니다.

In [ ]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('../../readonly/glove.6B.50d.txt')

다음의 데이터를 불러왔습니다.
- `word_to_index` : 어휘 세트에서 특정 단어와 단어의 index를 맵핑하는 딕셔너리
- `index_to_word` : index와 단어를 맵핑하는 딕셔너리
- `word_to_vec_map` : 단어와 GloVe 벡터 표현을 맵핑하는 딕셔너리

아래 코드를 통해 위의 함수가 어떻게 동작하는지 확인해봅시다.

In [ ]:
word = "cucumber"
idx = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(idx) + "th word in the vocabulary is", index_to_word[idx])

**연습 문제** : `sentence_to_avg()` 함수를 구현하세요. 두 단계를 따라야 합니다.
1. 모든 문장을 소문자로 바꾸세요. 그리고 문장을 단어들의 리스트로 쪼개야 합니다.
  - `X.lower()` 함수와 `X.split()` 함수를 사용하세요.
2. 각 단어마다, GloVe 벡터 값을 받아와서 다음을 수행하세요.
  - 모든 GloVe word vector에 대한 평균을 계산하세요.
  - `numpy.zeros()` 함수를 사용하세요.

#### 추가 힌트
- 0으로 구성된 `avg` 배열을 생성할 때, `word_to_vec_map` 에 있는 다른 word vector와 동일한 shape의 벡터가 되길 원할 것입니다.
  - `word_to_vec_map`에 있는 단어를 선택하고 `.shape` 필드를 사용할 수 이습니다.
  - 특정 단어에 엑세스할 때 해당 단어를 하드코딩하지 않도록 주의하세요. 다시 말해, 이 과제에 `word_to_vec_map`에 `the`라는 단어가 보인다면, 채점 알고리즘이 함수를 호출할 때 `the`라는 단어가 `word_to_vec_map`에 있을 것이라고 가정해선 안됩니다.
  - 힌트 : 입력 문장에서 검색한 word vector 중 하나를 사용하여 word vector의 shape를 찾을 수 있습니다.

In [ ]:
# GRADED FUNCTION: sentence_to_avg

def sentence_to_avg(sentence, word_to_vec_map):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
    """
    
    ### START CODE HERE ###
    # Step 1: Split sentence into list of lower case words (≈ 1 line)
    words = None

    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = None
    
    # Step 2: average the word vectors. You can loop over the words in the list "words".
    total = 0
    for w in None:
        total += None
    avg = None
    
    ### END CODE HERE ###
    
    return avg

In [ ]:
avg = sentence_to_avg("Morrocan couscous is my favorite dish", word_to_vec_map)
print("avg = \n", avg)

**모범 답안**:

```Python
avg =
[-0.008005    0.56370833 -0.50427333  0.258865    0.55131103  0.03104983
 -0.21013718  0.16893933 -0.09590267  0.141784   -0.15708967  0.18525867
  0.6495785   0.38371117  0.21102167  0.11301667  0.02613967  0.26037767
  0.05820667 -0.01578167 -0.12078833 -0.02471267  0.4128455   0.5152061
  0.38756167 -0.898661   -0.535145    0.33501167  0.68806933 -0.2156265
  1.797155    0.10476933 -0.36775333  0.750785    0.10282583  0.348925
 -0.27262833  0.66768    -0.10706167 -0.283635    0.59580117  0.28747333
 -0.3366635   0.23393817  0.34349183  0.178405    0.1166155  -0.076433
  0.1445417   0.09808667]
```

#### Model

이제 `model()` 함수를 구현하기 위한 모든 조각들을 완성했습니다. `sentence_to_avg()` 함수를 사용하고 난 다음에, 여러분은 다음의 작업을 수행해야 합니다.
- 평균값을 forward propagation에 전달하세요.
- 비용을 계산합니다.
- softmax 파라미터를 업데이트하기 위한 back propagation을 수행합니다.

**연습 문제**: 그림 2번에 표현된 `model()` 함수를 구현하세요.
- forward pass 및 크로스 엔트로피 손실함수를 구현하기 위해 필요한 공식은 다음과 같습니다.
- $Y_{oh}$("Y one hot") 변수는 라벨링된 출력값의 one-hot 인코딩 벡터 값입니다.

$$ z^{(i)} = W . avg^{(i)} + b$$

$$ a^{(i)} = softmax(z^{(i)})$$

$$ \mathcal{L}^{(i)} = - \sum_{k = 0}^{n_y - 1} Y_{oh,k}^{(i)} * log(a^{(i)}_k)$$

**참고** : 보다 효율적인 벡터화 된 구현을 사용할 수 있습니다. 지금은 알고리즘을 더 잘 이해하고 디버깅을 더 쉽게하기 위해 중첩 된 for 루프를 사용하겠습니다.

위에서 불러온 함수 `softmax()`를 사용하세요.

In [ ]:
# GRADED FUNCTION: model

def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations = 400):
    """
    Model to train word vector representations in numpy.
    
    Arguments:
    X -- input data, numpy array of sentences as strings, of shape (m, 1)
    Y -- labels, numpy array of integers between 0 and 7, numpy-array of shape (m, 1)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    learning_rate -- learning_rate for the stochastic gradient descent algorithm
    num_iterations -- number of iterations
    
    Returns:
    pred -- vector of predictions, numpy-array of shape (m, 1)
    W -- weight matrix of the softmax layer, of shape (n_y, n_h)
    b -- bias of the softmax layer, of shape (n_y,)
    """
    
    np.random.seed(1)

    # Define number of training examples
    m = Y.shape[0]                          # number of training examples
    n_y = 5                                 # number of classes  
    n_h = 50                                # dimensions of the GloVe vectors 
    
    # Initialize parameters using Xavier initialization
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    # Convert Y to Y_onehot with n_y classes
    Y_oh = convert_to_one_hot(Y, C = n_y) 
    
    # Optimization loop
    for t in range(num_iterations): # Loop over the number of iterations
        for i in range(m):          # Loop over the training examples
            
            ### START CODE HERE ### (≈ 4 lines of code)
            # Average the word vectors of the words from the i'th training example
            avg = None

            # Forward propagate the avg through the softmax layer
            z = None
            a = None

            # Compute cost using the i'th training label's one hot representation and "A" (the output of the softmax)
            cost = None
            ### END CODE HERE ###
            
            # Compute gradients 
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz

            # Update parameters with Stochastic Gradient Descent
            W = W - learning_rate * dW
            b = b - learning_rate * db
        
        if t % 100 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            pred = predict(X, Y, W, b, word_to_vec_map) #predict is defined in emo_utils.py

    return pred, W, b

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(np.eye(5)[Y_train.reshape(-1)].shape)
print(X_train[0])
print(type(X_train))
Y = np.asarray([5,0,0,5, 4, 4, 4, 6, 6, 4, 1, 1, 5, 6, 6, 3, 6, 3, 4, 4])
print(Y.shape)

X = np.asarray(['I am going to the bar tonight', 'I love you', 'miss you my dear',
 'Lets go party and drinks','Congrats on the new job','Congratulations',
 'I am so happy for you', 'Why are you feeling bad', 'What is wrong with you',
 'You totally deserve this prize', 'Let us go play football',
 'Are you down for football this afternoon', 'Work hard play harder',
 'It is suprising how people can be dumb sometimes',
 'I am very disappointed','It is the best day in my life',
 'I think I will end up alone','My life is so boring','Good job',
 'Great so awesome'])

print(X.shape)
print(np.eye(5)[Y_train.reshape(-1)].shape)
print(type(X_train))


다음 셀을 실행하여 모델을 훈련하고 softmax 파라미터 (W, b)를 학습합니다.

In [ ]:
pred, W, b = model(X_train, Y_train, word_to_vec_map)
print(pred)

**모범 답안** (on a subset of iterations):

<table>
    <tr>
        <td>
            **Epoch: 0**
        </td>
        <td>
           cost = 1.95204988128
        </td>
        <td>
           Accuracy: 0.348484848485
        </td>
    </tr>
<tr>
        <td>
            **Epoch: 100**
        </td>
        <td>
           cost = 0.0797181872601
        </td>
        <td>
           Accuracy: 0.931818181818
        </td>
    </tr>
<tr>
        <td>
            **Epoch: 200**
        </td>
        <td>
           cost = 0.0445636924368
        </td>
        <td>
           Accuracy: 0.954545454545
        </td>
    </tr>
    <tr>
        <td>
            **Epoch: 300**
        </td>
        <td>
           cost = 0.0343226737879
        </td>
        <td>
           Accuracy: 0.969696969697
        </td>
    </tr>
</table>

훌륭합니다! 모델은 훈련 세트에서 매우 높은 정확도를 가지고 있습니다. 이제 테스트 세트에서 어떻게 작동하는지 살펴 보겠습니다.

### 1.4 - Examining test set performance
- `em_util.spy`에 정의되어 있는 `predict` 함수를 사용합니다.

In [ ]:
print("Training set:")
pred_train = predict(X_train, Y_train, W, b, word_to_vec_map)
print('Test set:')
pred_test = predict(X_test, Y_test, W, b, word_to_vec_map)

**모범 답안**:

<table>
    <tr>
        <td>
            **Train set accuracy**
        </td>
        <td>
           97.7
        </td>
    </tr>
    <tr>
        <td>
            **Test set accuracy**
        </td>
        <td>
           85.7
        </td>
    </tr>
</table>

* 무작위 추측은 5 개의 클래스가 있는 경우 20% 의 정확도를 가집니다. (1/5 = 20 %).
* 127 개의 예제에 대한 훈련한 것에 비해 꽤나 좋은 성능입니다.

#### The model matches emojis to relevant words

훈련 세트에서, 알고리즘은 다음 문장을 받아서
> "I love you"

다음의 이모지를 출력할 수 있습니다.
> ❤️

- 트레이닝 세트에 "adore"라는 단어가 없다는 것을 알 수 있습니다.
- 그럼에도 불구하고 "I adore you"라는 문장을 입력하면 어떻게 되는지 확인해봅시다.

In [ ]:
X_my_sentences = np.array(["i adore you", "i love you", "funny lol", "lets play with a ball", "food is ready", "not feeling happy"])
Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

pred = predict(X_my_sentences, Y_my_labels , W, b, word_to_vec_map)
print_predictions(X_my_sentences, pred)

놀랍습니다!


* adore에는 love와 유사한 임베딩이 있기 때문에 알고리즘은 이전에 본 적이 없는 단어로도 올바른 값을 출력합니다.
* heart, dear, beloved 또는 adore와 같은 단어에는 love와 유사한 임베딩 벡터가 있습니다.
  * 위의 입력을 자유롭게 수정하고 다양한 입력 문장을 시도해보세요.
  * 얼마나 잘 작동합니까?

#### Word ordering isn't considered in this model

* 모델은 다음 문장에 대한 올바른 결과를 얻을 수 없습니다.
> "not feeling happy"

* 이 알고리즘은 단어 순서를 무시하므로, "not happy"와 같은 구문을 이해는데 적합하지 않습니다.



#### Confusion matrix

- Confusion matrix를 출력하면 모델에서 어떤 클래스가 더 어려운지 이해하는데 도움이 됩니다.
- Confusion matrix는 레이블이 하나의 클래스("실제" 클래스)인 예제가, 다른 클래스("예측된" 클래스)의 알고리즘에 의해 얼마나 자주 잘못 표시되는지를 보여줍니다.

In [ ]:
print(Y_test.shape)
print('           '+ label_to_emoji(0)+ '    ' + label_to_emoji(1) + '    ' +  label_to_emoji(2)+ '    ' + label_to_emoji(3)+'   ' + label_to_emoji(4))
print(pd.crosstab(Y_test, pred_test.reshape(56,), rownames=['Actual'], colnames=['Predicted'], margins=True))
plot_confusion_matrix(Y_test, pred_test)

## What you should remember from this section

-127 개의 데이터 세트로도 Emojifying을 수행하는 좋은 모델을 얻을 수 있습니다.
  - 이것은 일반화를 위한 강력한 word vector를 사용하기 때문입니다.
- Emojify-V1은 "This movie is not good and not enjoyable"과 같은 문장에서 성능이 떨어집니다.
  - 단어의 조합을 이해하지 못합니다.
  - 단어의 순서를 고려하지 않고 모든 단어의 embedding 벡터를 평균화합니다.
    
**다음 섹션에서 더 나은 알고리즘을 만들 것입니다!**

## 2 - Emojifier-V2 : Using LSTMs in Keras

단어 **시퀀스**를 입력으로 사용하는 LSTM 모델을 구축해 보겠습니다!
* 이 모델은 단어의 순서를 고려할 수 있습니다.
* Emojifier-V2는 사전 학습 된 단어 임베딩을 계속 사용하여 단어를 나타냅니다.
* LSTM에 word embedding을 제공합니다.
* LSTM은 가장 적합한 이모티콘을 예측하는 방법을 학습합니다.

다음 셀을 실행하여 Keras 패키지를로드하십시오.

In [ ]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

### 2.1 - Overview of the model

아래는 여러분이 구현할 Emojifier-v2 모델의 구조입니다.

<img src="arts/emojifier-v2.png" style="width:700px;height:400px;">
<center>그림 3 : Emojfifier-V2. 2-layer의 LSTM 시퀸스 분류 모델</center>


### 2.2 Keras and mini-batching

- 이번 과제에서는, 미니 배치를 사용해 케라스 모델을 훈련하고자 합니다.
- 하지만 대부분의 딥 러닝 프레임워크는 동일한 미니 배치의 모든 스퀸스가 동일한 길이를 가져야 합니다.
  - 이를 통해 벡터화가 가능해집니다 : 3개 단어 문장과 4개 단어 문장이 있는 경우 필요한 계산이 다릅니다(하나는 LSTM의 3단계, 하나는 LSTM의 4단계를 거쳐야 합니다). 둘 다 동시에 할 수 있습니다.

#### Padding handles sequences of varying length

* 다른 길이의 시퀀스를 처리하는 일반적인 해결책은 패딩을 사용하는 것입니다. 구체적으로 특별히:
  * 최대 시퀀스 길이 설정
  * 동일한 길이를 갖도록 모든 시퀀스를 채 웁니다.

##### Example of padding

* 최대 시퀀스 길이가 20 인 경우 각 입력 문장의 길이가 20이되도록 모든 문장을 "0"으로 채울 수 있습니다.
* 따라서 "I love you"라는 문장은 $ (e_ {I}, e_ {love}, e_ {you}, \vec {0}, \vec {0}, \ldots, \vec {0 }) $.
*이 예에서는 20 단어보다 긴 문장은 잘 려야합니다.
* 최대 시퀀스 길이를 선택하는 한 가지 방법은 훈련 세트에서 가장 긴 문장의 길이를 선택하는 것입니다.

### 2.3 - The Embedding layer

- Keras에서, embedding matrix는 "layer" 라는 단어로 표현됩니다.
- Embedding matrix는 단어 index와 embedding 벡터를 맵핑합늬다.
  - 단어 인덱스는 양의 정수입니다.
  - Embedding vector는 고정된 사이즈의 dense 벡터 입니다.
  - 특정 벡터가 "dense" 하다는 것은, 벡터 내의 대부분의 값이 0이 아니라는 것을 의미합니다. 따라서 이와 반대되는 "dense"가 아닌 벡터의 예시로 one-hot 인코딩 벡터를 들 수 있습니다.
- Embedding matrix는 두 가지 방법으로 구할 수 있습니다.
  - 데이터를 긁어 모아, 직접 embedding을 학습하여 구합니다.
  - 사전에 훈련된 embedding 벡터를 사용합니다.

#### Using and updateing pre-trained embeddings

- 이 파트에서, 여러분은 케라스를 사용해 [Embedding()](https://keras.io/layers/embeddings/) 레이어를 만드는 방법을 배우게 됩니다.
- 먼저 Embedding 레이어를 50 차원의 GloVe 벡터로 초기화합니다.
- 아래 코드에서 Keras를 사용하여 이 레이어를 훈련하거나 고정 된 상태로 두는 방법을 보여 드리겠습니다.
- 훈련 세트가 매우 작기 때문에 GloVe 임베딩을 업데이트하는 대신 고정 된 상태로 둡니다.

#### Inputs and outputs to the embedding layer


- `Embedding ()`레이어의 입력은 **(배치 크기, 최대 입력 길이)** 크기의 정수 행렬입니다.
  - 이 입력은 인덱스 목록 (정수)으로 변환 된 문장에 해당합니다.
  - 입력에서 가장 큰 정수 (가장 높은 단어 인덱스)는 어휘 크기보다 크지 않아야합니다.
- Embedding 레이어는 (배치 크기, 최대 입력 길이, 단어 벡터의 차원) shape의 배열을 출력합니다.

- 그림은 임베딩 레이어를 통해 두 개의 예제 문장이 전파되는 모습입니다.
  - 두 예제 모두`max_len = 5` 길이로 zero padding되었습니다.
  - 단어 임베딩은 길이가 50 단위입니다.
  - shape는 `(2, max_len, 50)`입니다.

#### Prepare to input sentenses

**연습 문제**:
- 아무 문장 (X)를 받아서 embedding layer의 입력으로 변환시켜주는 `sentences_to_indices` 함수를 완성하세요.
  - 각 훈련용 문장들을 인덱스(각 인덱스들은 문장에서의 각 단어에 대응됩니다)로 변환하세요.
  - 가장 긴 문장의 길이에 맞게 모든 리스트들을 zero padding 하세요.

##### 추가 힌트
- 반복문에서 `enumerate()`를 함수 사용할 수 있지만 자동 채점 알고리즘에 원활한 값을 전달하려면 j를 명시 적으로 초기화하고 증가 시켜가며 코드를 작성하세요.

In [ ]:
for idx, val in enumerate(["I", "like", "learning"]):
    print(idx,val)

In [ ]:
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = None
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words =None
        
        # Initialize j to 0
        j = None
        
        # Loop over the words of sentence_words
        for w in None:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = None
            # Increment j to j + 1
            j = None
            
    ### END CODE HERE ###
    
    return X_indices

다음 셀을 실행하여 `sentences_to_indices()`의 기능을 확인하고 결과를 확인합니다.

In [ ]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =\n", X1_indices)

**모범 답안**:

```Python
X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices =
 [[ 155345.  225122.       0.       0.       0.]
 [ 220930.  286375.   69714.       0.       0.]
 [ 151204.  192973.  302254.  151349.  394475.]]
```

#### Build embedding layer
- 이제 케라스와, 사전 훈련된 word-vector를 사용해 `Embedding()` 레이어를 만들어 보겠습니다.
- Embedding 레이어는 단어 인덱스 리스트를 입력으로 받습니다.
  - `sentences_to_indices()` 함수는 이 단어 인덱스 리스트를 생성합니다.
- Embedding 레이어는 해당 문장의 word embedding을 리턴합니다.

**Exercise** : 아래 단계에 따라, `pretrained_embedding layer()` 함수를 구현하세요.

1. Numpy array인 embedding matrix를 0으로 초기화하세요.
  - 이 embedding matrix의 열(row)은 각 어휘 세트에서 유일한 한 개의 단어입니다.
    - 추가적으로 "unknown" word라고 하는 열이 있습니다.
    - 따라서 `vocab_len` 값은 어휘세트의 단어의 개수에, 1을 더한 값입니다.
  - 각 열은 단어의 벡터 표현을 담고 있습니다.
    - 예를 들어 GloVe 단어 벡터를 사용한다면 한 개의 열이 50개의 포지션을 갖고 있을 것입니다.
  - 아래 코드에서, `emb_dim`은 word embedding의 길이를 나타냅니다.
2. Embedding matrix의 각 행을 단어의 벡터 표현으로 채웁니다.
  - `word_to_index` 각 단어는 문자열입니다.
  - `word_to_vec_map`은 key가 문자열이고, value가 단어 벡터인 딕셔너리입니다.
3. Keras Embedding layer를 정의합니다.
  - [Embedding()](https://keras.io/layers/embeddings/)을 사용합니다.
  - 입력 차원은 어휘의 길이(고유 단어 수에 1을 더한 값)입니다.
  - 이 레이어의 embedding을 고정시킵니다.
    - `trainable = True`로 설정하면, 최적화 알고리즘이 word embedding 값을 수정할 수 있습니다.
    - 이 경우 우리는 모델이 word embedding 값을 변경하지 않길 원합니다.
4. Word embedding과 같도록 embedding의 가중치를 설정합니다.
  - 이는 이미 완성된 코드의 일부라서, 별도로 수정할 필요가 없습니다.

In [ ]:
# GRADED FUNCTION: pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Step 1
    # Initialize the embedding matrix as a numpy array of zeros.
    # See instructions above to choose the correct shape.
    emb_matrix = None
    
    # Step 2
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = None

    # Step 3
    # Define Keras embedding layer with the correct input and output sizes
    # Make it non-trainable.
    embedding_layer = None
    ### END CODE HERE ###

    # Step 4 (already done for you; please do not modify)
    # Build the embedding layer, it is required before setting the weights of the embedding layer. 
    embedding_layer.build((None,)) # Do not modify the "None".  This line of code is complete as-is.
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

**모범 답안**:

```Python
weights[0][1][3] = -0.3403
```

## 2.3 Building the Emojifier-V2


이제 Emojifier-V2 모델을 빌드 해 보겠습니다.
* Embedding 레이어의 출력을 LSTM 네트워크에 공급합니다.

<img src="arts/emojifier-v2.png" style="width:700px;height:400px;">
<center>그림 3 : Emojifier-v2. 2 계층 LSTM 시퀀스 분류 모델</center>


**연습 문제** : 그림 3에 표시된 아키텍처의 Keras 그래프를 작성하는 `Emojify_V2()`를 구현합니다.
* 모델은 `input_shape`로 정의 된 shape (`m`,`max_len`,)의 문장 배열을 입력으로받습니다.
* 모델은 (`m`,`C = 5`) shape의 softmax 확률 벡터를 출력합니다.

* 다음 Keras 레이어를 사용해야 할 수 있습니다.
    * [input()](https://keras.io/layers/core/#input)
        * `shape` 및 `dtype` 매개 변수를 설정합니다.
        * 입력은 정수이므로 데이터 유형을 문자열 'int32'로 지정할 수 있습니다.
    * [LSTM()](https://keras.io/layers/recurrent/#lstm)
        * `units` 및`return_sequences` 매개 변수를 설정합니다.
    * [Dropout()](https://keras.io/layers/core/#dropout)
        * `rate` 매개 변수를 설정합니다.
    * [Dense()](https://keras.io/layers/core/#dense)
        * '단위'설정,
        * `Dense()`에는 `activation`매개 변수가 있습니다. 자동 채점기에 올바른 값을 전달하기 위해 `Dense()`내에 활성화를 설정하지 마십시오. 이를 위해 별도의 '활성화' 레이어를 사용합니다.
    * [Activation()](https://keras.io/activations/).
        * 선택한 활성화를 소문자 문자열로 전달할 수 있습니다.
    * [Model](https://keras.io/models/model/)
        '입력'과 '출력'을 설정합니다.


#### 추가 힌트
* 이러한 Keras 레이어는 개체를 반환하고 이전 레이어의 출력을 해당 개체에 대한 입력 인수로 제공한다는 점을 기억하십시오. 반환 된 객체는 동일한 줄에서 생성되고 호출 될 수 있습니다.

  ```python
  # How to use Keras layers in two lines of code
  dense_object = Dense(units = ...)
  X = dense_object(inputs)

  # How to use Keras layers in one line of code
  X = Dense(units = ...)(inputs)
  ```

* `pretrained_embedding_layer`가 반환하는`embedding_layer`는 단일 인수 (문장 인덱스)를 전달하여 함수로 호출 할 수있는 레이어 객체입니다.

* 다음은 문제가 발생할 경우를 대비 한 샘플 코드입니다.
  ```python
  raw_inputs = Input(shape=(maxLen,), dtype='int32')
  preprocessed_inputs = ... # some pre-processing
  X = LSTM(units = ..., return_sequences= ...)(processed_inputs)
  X = Dropout(rate = ..., )(X)
  ...
  X = Dense(units = ...)(X)
  X = Activation(...)(X)
  model = Model(inputs=..., outputs=...)
  ...
  ```

In [ ]:
# GRADED FUNCTION: Emojify_V2

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph.
    # It should be of shape input_shape and dtype 'int32' (as it contains indices, which are integers).
    sentence_indices = None
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = None
    
    # Propagate sentence_indices through your embedding layer
    # (See additional hints in the instructions).
    embeddings = None   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # The returned output should be a batch of sequences.
    X = None
    # Add dropout with a probability of 0.5
    X = None
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = None
    # Add dropout with a probability of 0.5
    X = None
    # Propagate X through a Dense layer with 5 units
    X = None
    # Add a softmax activation
    X = None
    
    # Create Model instance which converts sentence_indices into X.
    model = None
    
    ### END CODE HERE ###
    
    return model

다음 셀을 실행하여 모델을 만들고 요약을 확인합니다. 데이터 세트의 모든 문장이 10 단어 미만이므로 `max_len = 10`을 선택했습니다. 아키텍처가 표시되어야 합니다. "20,223,927" 개의 파라미터를 사용합니다. 이 중 20,000,050(Embedding이라는 단어)은 학습 할 수 없고 나머지 223,877개 의 파라미터를 학습할 수 있습니다. 우리의 어휘 크기는 400,001 단어 (0 ~ 400,000의 유효한 인덱스 포함)가 있기 때문에 400,001 \ * 50 = 20,000,050 개의 훈련 불가능한 파라미터가 있습니다.

In [ ]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

평소와 같이 Keras에서 모델을 생성 한 후에는 모델을 컴파일하고 사용할 손실, 최적화 도구 및 메트릭을 정의해야합니다. categorical_crossentropy loss, adam Optimizer 및 [ 'accuracy'] 측정 항목을 사용하여 모델을 컴파일합니다.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

모델을 훈련 할 시간입니다. Emojifier-V2 `model`은 shape (`m`,`max_len`)의 배열을 입력으로 취하고 (`m`,`number of classes`) shape의 확률 벡터를 출력합니다. 따라서 X_train (문자열로 된 문장 배열)을 X_train_indices (단어 색인 목록으로 문장 배열)로, Y_train (인덱스로 레이블)을 Y_train_oh (원-핫 벡터로 레이블)로 변환해야합니다.

In [ ]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)


Keras 모델을 `X_train_indices` 및 `Y_train_oh`에 맞춥니다. `epochs = 50` 과 `batch_size = 32`를 사용합니다.

In [ ]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

모델은 학습 세트에서 약 ** 90 % ~ 100 % 정확도 **를 수행해야합니다. 당신이 얻는 정확한 정확도는 약간 다를 수 있습니다. 테스트 세트에서 모델을 평가하려면 다음 셀을 실행하십시오.

In [ ]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)


테스트 정확도는 80 %에서 95 % 사이 여야합니다. 아래 셀을 실행하여 라벨이 잘못 지정된 예를 확인하세요.

In [ ]:
# This code allows you to see the mislabelled examples
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

이제 직접 다른 단어를 사용해서 시도해 볼 수 있습니다. 아래에 자신의 문장을 작성하십시오.

In [ ]:
# Change the sentence below to see your prediction. Make sure all the words are in the Glove embeddings.  
x_test = np.array(['not feeling happy'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

## LSTM version accounts for word order

* 이전에는 Emojify-V1 모델이 "not feeling happy"라는 레이블을 올바르게 표시하지 않았지만 Emojiy-V2 구현에서는 해당 내용이 올바르게 작동했습니다.
  * (Keras의 출력은 매번 약간 무작위이므로 동일한 결과를 얻지 못할 수 있습니다.)
* 현재 모델은 여전히 ​​부정 표현을 이해하는 데 매우 강력하지 않습니다 (예 : "not happy").
  * 이것은 훈련 세트가 작고 부정 표현의 예가 많지 않기 때문입니다.
  * 그러나 훈련 세트가 더 크면 LSTM 모델이 이러한 복잡한 문장을 이해하는 데 Emojify-V1 모델보다 훨씬 낫습니다.

### Congratulations!

이 과제를 완료했습니다! ❤️❤️❤️


## What you should remember
- 훈련 세트가 작은 NLP 작업이있는 경우 word embedding을 사용하면 알고리즘에 크게 도움이 될 수 있습니다.
- Word embedding을 사용하면 모델이 학습 세트에 나타나지 않을 수도 있는 테스트 세트의 단어에 대해 작업 할 수 있습니다.
- Keras (및 대부분의 다른 딥 러닝 프레임 워크)의 훈련 시퀀스 모델에는 몇 가지 중요한 세부 정보가 필요합니다.
  - 미니 배치를 사용하려면 미니 배치의 모든 예제가 **동일한 길이**를 갖도록 시퀀스를 **패딩**해야합니다.
  - `Embedding()` 레이어는 사전 훈련 된 값으로 초기화 할 수 있습니다.
    - 이러한 값은 고정되거나 데이터 세트에서 추가 학습 될 수 있습니다.
    - 그러나 라벨이 지정된 데이터 세트가 작다면 일반적으로 사전 학습 된 대규모 임베딩 세트를 학습시킬 가치가 없습니다.
    - `LSTM ()`에는`return_sequences`라는 플래그가있어 모든 숨겨진 상태를 반환할지 아니면 마지막 상태 만 반환 할지를 결정합니다.
    - `LSTM ()`바로 뒤에`Dropout ()`을 사용하여 네트워크를 정규화 할 수 있습니다.

#### Input sentences:
```Python
"Congratulations on finishing this assignment and building an Emojifier."
"We hope you're happy with what you've accomplished in this notebook!"
```
#### Output emojis:
# 😀😀😀😀😀😀

## Acknowledgments

Thanks to Alison Darcy and the Woebot team for their advice on the creation of this assignment. 
* Woebot is a chatbot friend that is ready to speak with you 24/7. 
* Part of Woebot's technology uses word embeddings to understand the emotions of what you say. 
* You can chat with Woebot by going to http://woebot.io

<img src="arts/woebot.png" style="width:600px;height:300px;">